In [72]:
import sqlalchemy as sa
from datetime import timedelta,datetime
import pandas as pd
import numpy as np
import requests
import json
import time

In [73]:
#Database Credentials
my_db_secret = {
    'drivername': 'postgresql+psycopg2', 
    'host': 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port': '5432', 
    'username': 'sushmit9', 
    'password': '2023!Schulich',
    'database': 'sushmit9_db'
}

#Creating Databse URL
my_db_url = sa.engine.URL.create(
    drivername = my_db_secret['drivername'],
    username = my_db_secret['username'],
    password = my_db_secret['password'],
    host = my_db_secret['host'],
    port = my_db_secret['port'],
    database = my_db_secret['database'])

# Create the SQLAlchemy engine
my_db_engine = sa.create_engine(my_db_url)

# Current Weather for Toronto

In [74]:
# Fetching data
# Data Source: https://openweathermap.org/current
lat= 43.6532
lon = -79.3832
api_key = "7bada83efe23d6a1b6204c7ee293a2f4"
api_url = 'https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid={}'.format(lat, lon, api_key)
data = requests.get(api_url)
data = data.json()
data    

{'coord': {'lon': -79.3832, 'lat': 43.6532},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 302,
  'feels_like': 303.24,
  'temp_min': 298.35,
  'temp_max': 303.91,
  'pressure': 1004,
  'humidity': 55},
 'visibility': 10000,
 'wind': {'speed': 2.68, 'deg': 266, 'gust': 5.36},
 'clouds': {'all': 45},
 'dt': 1691869497,
 'sys': {'type': 1,
  'id': 718,
  'country': 'CA',
  'sunrise': 1691835512,
  'sunset': 1691886411},
 'timezone': -14400,
 'id': 6167863,
 'name': 'Downtown Toronto',
 'cod': 200}

#### Feature Engineering

In [75]:
# Flatten the nested dictionaries
flattened_data = {
    'lon': data['coord']['lon'],
    'lat': data['coord']['lat'],
    'weather_id': data['weather'][0]['id'],
    'weather_main': data['weather'][0]['main'],
    'weather_description': data['weather'][0]['description'],
    'weather_icon': data['weather'][0]['icon'],
    'base': data['base'],
    'temp': data['main']['temp'],
    'feels_like': data['main']['feels_like'],
    'temp_min': data['main']['temp_min'],
    'temp_max': data['main']['temp_max'],
    'pressure': data['main']['pressure'],
    'humidity': data['main']['humidity'],
    'visibility': data['visibility'],
    'wind_speed': data['wind']['speed'],
    'wind_deg': data['wind']['deg'],
    'clouds_all': data['clouds']['all'],
    'dt': data['dt'],
    'sys_type': data['sys']['type'],
    'sys_id': data['sys']['id'],
    'sys_country': data['sys']['country'],
    'sys_sunrise': data['sys']['sunrise'],
    'sys_sunset': data['sys']['sunset'],
    'timezone': data['timezone'],
    'city_id': data['id'],
    'city_name': data['name'],
    'cod': data['cod']
    }

# Convert the flattened data to a DataFrame
df = pd.DataFrame(flattened_data, index=[0])
df

,lon,lat,weather_id,weather_main,weather_description,weather_icon,base,temp,feels_like,temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,city_id,city_name,cod
0,-79.3832,43.6532,802,Clouds,scattered clouds,03d,stations,302,303.24,298.35,...,1691869497,1,718,CA,1691835512,1691886411,-14400,6167863,Downtown Toronto,200


In [105]:
df.dtypes

dt                       int64
temp                   float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
sea_level                int64
grnd_level               int64
humidity                 int64
weather_id               int64
weather_main            object
weather_description     object
weather_icon            object
clouds_all               int64
wind_speed             float64
wind_deg                 int64
gust                   float64
visibility               int64
pop                    float64
sys_pod                 object
dt_txt                  object
city_id                  int64
city_name               object
city_lat               float64
city_lon               float64
city_country            object
city_timezone            int64
city_sunrise             int64
city_sunset              int64
dtype: object

In [50]:
# Convert UNIX timestamps to 'hh:mm:ss' format using lambda function
df['dt'] = df['dt'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
df['sys_sunrise'] = df['sys_sunrise'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
df['sys_sunset'] = df['sys_sunset'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
# Convert timezone shift to 'hhmm' format using lambda function
df['timezone'] = df['timezone'].apply(lambda shift: ('-' if shift < 0 else '') + '{:02d}{:02d}'.format(abs(shift) // 3600, (abs(shift) % 3600) // 60))

df

,lon,lat,weather_id,weather_main,weather_description,weather_icon,base,temp,feels_like,temp_min,...,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,timezone,city_id,city_name,cod
0,-79.3839,43.6535,803,Clouds,broken clouds,04d,stations,300.71,302.46,298.04,...,18:47:05,1,718,CA,10:18:32,00:26:51,-0400,6167863,Downtown Toronto,200


In [51]:
df.columns

Index(['lon', 'lat', 'weather_id', 'weather_main', 'weather_description',
       'weather_icon', 'base', 'temp', 'feels_like', 'temp_min', 'temp_max',
       'pressure', 'humidity', 'visibility', 'wind_speed', 'wind_deg',
       'clouds_all', 'dt', 'sys_type', 'sys_id', 'sys_country', 'sys_sunrise',
       'sys_sunset', 'timezone', 'city_id', 'city_name', 'cod'],
      dtype='object')

In [52]:
#Dropping unneceassry columns and intenal parameters
df.drop(['weather_id','weather_icon','base','sys_type','sys_id', 'cod'], axis=1, inplace=True)
df

,lon,lat,weather_main,weather_description,temp,feels_like,temp_min,temp_max,pressure,humidity,...,wind_speed,wind_deg,clouds_all,dt,sys_country,sys_sunrise,sys_sunset,timezone,city_id,city_name
0,-79.3839,43.6535,Clouds,broken clouds,300.71,302.46,298.04,303.13,1004,65,...,3.6,160,75,18:47:05,CA,10:18:32,00:26:51,-0400,6167863,Downtown Toronto


#### DATA INGESTION PIPELINE

In [57]:
# Function to fetch and ingest data - compiling everything above into a function
def fetch_and_ingest_data():
    lat= 43.6532
    lon = -79.3832
    api_key = "7bada83efe23d6a1b6204c7ee293a2f4"
    api_url = 'https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid={}'.format(lat, lon, api_key)
    data = requests.get(api_url)
    data = data.json()
    
    # Flatten the nested dictionaries
    flattened_data = {
    'lon': data['coord']['lon'],
    'lat': data['coord']['lat'],
    'weather_id': data['weather'][0]['id'],
    'weather_main': data['weather'][0]['main'],
    'weather_description': data['weather'][0]['description'],
    'weather_icon': data['weather'][0]['icon'],
    'base': data['base'],
    'temp': data['main']['temp'],
    'feels_like': data['main']['feels_like'],
    'temp_min': data['main']['temp_min'],
    'temp_max': data['main']['temp_max'],
    'pressure': data['main']['pressure'],
    'humidity': data['main']['humidity'],
    'visibility': data['visibility'],
    'wind_speed': data['wind']['speed'],
    'wind_deg': data['wind']['deg'],
    'clouds_all': data['clouds']['all'],
    'dt': data['dt'],
    'sys_type': data['sys']['type'],
    'sys_id': data['sys']['id'],
    'sys_country': data['sys']['country'],
    'sys_sunrise': data['sys']['sunrise'],
    'sys_sunset': data['sys']['sunset'],
    'timezone': data['timezone'],
    'city_id': data['id'],
    'city_name': data['name'],
    'cod': data['cod']
    }
    
    # Convert the flattened data to a DataFrame
    df = pd.DataFrame(flattened_data, index=[0])

    # Convert UNIX timestamps to 'hh:mm:ss' format using lambda function
    df['dt'] = df['dt'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    df['sys_sunrise'] = df['sys_sunrise'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    df['sys_sunset'] = df['sys_sunset'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    
    # Convert timezone shift to 'hhmm' format using lambda function
    df['timezone'] = df['timezone'].apply(lambda shift: ('-' if shift < 0 else '') + '{:02d}{:02d}'.format(abs(shift) // 3600, (abs(shift) % 3600) // 60))

    
    #Dropping unneceassry columns and intenal parameters
    df.drop(['weather_id','weather_icon','base','sys_type','sys_id', 'cod'], axis=1, inplace=True)

    #Pushing dataframe to Database Schema
    df.to_sql(
    name      = 'Toronto_current_weather',
    con       = my_db_engine,
    schema    = 'Uploads',
    if_exists = 'replace',
    index     = False,
    method    = 'multi',
    dtype     = {
        'lon'                 : sa.types.FLOAT,
        'lat'                 : sa.types.FLOAT,
        'weather_main'        : sa.types.VARCHAR(30),
        'weather_description' : sa.types.VARCHAR(30),
        'temp'                : sa.types.FLOAT,
        'feels_like'          : sa.types.FLOAT,
        'temp_min'            : sa.types.FLOAT,
        'temp_max'            : sa.types.FLOAT,
        'pressure'            : sa.types.INTEGER,
        'humidity'            : sa.types.INTEGER,
        'visibility'          : sa.types.INTEGER,
        'wind_speed'          : sa.types.FLOAT,
        'wind_deg'            : sa.types.INTEGER,
        'clouds_all'          : sa.types.INTEGER,
        'dt'                  : sa.types.VARCHAR(30),
        'sys_country'         : sa.types.VARCHAR(30),
        'sys_sunrise'         : sa.types.VARCHAR(30),
        'sys_sunset'          : sa.types.VARCHAR(30),
        'timezone'            : sa.types.VARCHAR(30),
        'city_id'             : sa.types.INTEGER,
        'city_name'           : sa.types.VARCHAR(30)
    }
    )

# Immediately run the data fetching and ingestion process
print("Running data fetching and ingestion process immediately...")
fetch_and_ingest_data()
print("Data fetching and ingestion process has been executed.")

Running data fetching and ingestion process immediately...
Data fetching and ingestion process has been executed.


In [9]:
# Schedule the data fetching and ingestion process to run daily from the next day onwards
# P.S. - This is meant to be executed in the background. 
def schedule_daily_task(hour, minute):
    while True:
        current_time = time.localtime()
        if current_time.tm_hour >= hour and current_time.tm_min >= minute:
            break
        time.sleep(60)  # Sleep for a minute

    while True:
        fetch_and_ingest_data()
        print("Data fetching and ingestion process has been scheduled and executed.")
        time.sleep(24 * 60 * 60)  # Sleep for a day

# Schedule the data fetching and ingestion process to run every day at 12:00 PM
schedule_daily_task(hour=12, minute=0)

Data fetching and ingestion process has been scheduled and executed.


# Weather Forecasts for Toronto

In [132]:
# Fetching data
# Data Source: https://openweathermap.org/api/hourly-forecast
lat2= 43.6532
lon2 = -79.3832
api_key2 = "583b9776c32850d28350562414730cb1"
api_url2 = 'https://pro.openweathermap.org/data/2.5/forecast/hourly?lat={}&lon={}&appid={}'.format(lat2, lon2, api_key2)
data2 = requests.get(api_url2)
data2 = data2.json()
data2   

{'cod': '200',
 'message': 0,
 'cnt': 96,
 'list': [{'dt': 1691877600,
   'main': {'temp': 301.11,
    'feels_like': 302.02,
    'temp_min': 298.99,
    'temp_max': 301.11,
    'pressure': 1004,
    'sea_level': 1004,
    'grnd_level': 995,
    'humidity': 55,
    'temp_kf': 2.12},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 31},
   'wind': {'speed': 6.81, 'deg': 240, 'gust': 9.25},
   'visibility': 10000,
   'pop': 0.58,
   'rain': {'1h': 0.15},
   'sys': {'pod': 'd'},
   'dt_txt': '2023-08-12 22:00:00'},
  {'dt': 1691881200,
   'main': {'temp': 300.35,
    'feels_like': 301.18,
    'temp_min': 298.41,
    'temp_max': 300.35,
    'pressure': 1004,
    'sea_level': 1004,
    'grnd_level': 995,
    'humidity': 56,
    'temp_kf': 1.94},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}],
   'clouds': {'all': 28},
   'wind': {'speed': 5.59, 'deg': 26

#### Feature Engineering

In [133]:
# Flatten the nested data
flat_data_list = []

for item in data2['list']:
    flat_data = {
        'dt': item['dt'],
        'temp': item['main']['temp'],
        'feels_like': item['main']['feels_like'],
        'temp_min': item['main']['temp_min'],
        'temp_max': item['main']['temp_max'],
        'pressure': item['main']['pressure'],
        'sea_level': item['main'].get('sea_level'),  # Check if key exists
        'grnd_level': item['main'].get('grnd_level'),  # Check if key exists
        'temp_kf': item['main'].get('temp_kf'),  # Check if key exists
        'humidity': item['main']['humidity'],
        'weather_id': item['weather'][0]['id'],
        'weather_main': item['weather'][0]['main'],
        'weather_description': item['weather'][0]['description'],
        'weather_icon': item['weather'][0]['icon'],
        'clouds_all': item['clouds']['all'],
        'wind_speed': item['wind']['speed'],
        'wind_deg': item['wind']['deg'],
        'gust': item['wind']['gust'],
        'visibility': item['visibility'],
        'pop': item['pop'],
        'sys_pod': item['sys']['pod'],
        'dt_txt': item['dt_txt'],
        'city_id': data2['city']['id'],
        'city_name': data2['city']['name'],
        'city_lat': data2['city']['coord']['lat'],
        'city_lon': data2['city']['coord']['lon'],
        'city_country': data2['city']['country'],
        'city_timezone': data2['city']['timezone'],
        'city_sunrise': data2['city']['sunrise'],
        'city_sunset': data2['city']['sunset']
    }
    flat_data_list.append(flat_data)

# Convert the flattened data to a DataFrame
df2 = pd.DataFrame(flat_data_list)

df2.head()

,dt,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,temp_kf,humidity,...,sys_pod,dt_txt,city_id,city_name,city_lat,city_lon,city_country,city_timezone,city_sunrise,city_sunset
0,1691877600,301.11,302.02,298.99,301.11,1004,1004,995,2.12,55,...,d,2023-08-12 22:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-14400,1691835512,1691886411
1,1691881200,300.35,301.18,298.41,300.35,1004,1004,995,1.94,56,...,d,2023-08-12 23:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-14400,1691835512,1691886411
2,1691884800,299.11,299.11,297.42,299.11,1005,1005,996,1.69,59,...,d,2023-08-13 00:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-14400,1691835512,1691886411
3,1691888400,297.10,297.27,295.96,297.10,1006,1006,996,1.14,66,...,n,2023-08-13 01:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-14400,1691835512,1691886411
4,1691892000,295.28,295.45,295.28,295.28,1007,1007,997,0.00,73,...,n,2023-08-13 02:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-14400,1691835512,1691886411


In [134]:
df2.columns

Index(['dt', 'temp', 'feels_like', 'temp_min', 'temp_max', 'pressure',
       'sea_level', 'grnd_level', 'temp_kf', 'humidity', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon', 'clouds_all',
       'wind_speed', 'wind_deg', 'gust', 'visibility', 'pop', 'sys_pod',
       'dt_txt', 'city_id', 'city_name', 'city_lat', 'city_lon',
       'city_country', 'city_timezone', 'city_sunrise', 'city_sunset'],
      dtype='object')

In [135]:
# Convert UNIX timestamps to 'hh:mm:ss' format using lambda function
df2['dt'] = df2['dt'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
df2['city_sunrise'] = df2['city_sunrise'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
df2['city_sunset'] = df2['city_sunset'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
# Convert timezone shift to 'hhmm' format using lambda function
df2['city_timezone'] = df2['city_timezone'].apply(lambda shift: ('-' if shift < 0 else '') + '{:02d}{:02d}'.format(abs(shift) // 3600, (abs(shift) % 3600) // 60))

df2.head(3)

,dt,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,temp_kf,humidity,...,sys_pod,dt_txt,city_id,city_name,city_lat,city_lon,city_country,city_timezone,city_sunrise,city_sunset
0,22:00:00,301.11,302.02,298.99,301.11,1004,1004,995,2.12,55,...,d,2023-08-12 22:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-0400,10:18:32,00:26:51
1,23:00:00,300.35,301.18,298.41,300.35,1004,1004,995,1.94,56,...,d,2023-08-12 23:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-0400,10:18:32,00:26:51
2,00:00:00,299.11,299.11,297.42,299.11,1005,1005,996,1.69,59,...,d,2023-08-13 00:00:00,6167863,Downtown Toronto,43.6532,-79.3832,CA,-0400,10:18:32,00:26:51


In [136]:
#Dropping unneceassry columns and intenal parameters
df2.drop(['temp_kf','weather_id','weather_icon','city_id'], axis=1, inplace=True)

In [137]:
df2.head(3)

,dt,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,weather_main,...,pop,sys_pod,dt_txt,city_name,city_lat,city_lon,city_country,city_timezone,city_sunrise,city_sunset
0,22:00:00,301.11,302.02,298.99,301.11,1004,1004,995,55,Rain,...,0.58,d,2023-08-12 22:00:00,Downtown Toronto,43.6532,-79.3832,CA,-0400,10:18:32,00:26:51
1,23:00:00,300.35,301.18,298.41,300.35,1004,1004,995,56,Clouds,...,0.54,d,2023-08-12 23:00:00,Downtown Toronto,43.6532,-79.3832,CA,-0400,10:18:32,00:26:51
2,00:00:00,299.11,299.11,297.42,299.11,1005,1005,996,59,Clouds,...,0.42,d,2023-08-13 00:00:00,Downtown Toronto,43.6532,-79.3832,CA,-0400,10:18:32,00:26:51


In [138]:
df2.dtypes

dt                      object
temp                   float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
sea_level                int64
grnd_level               int64
humidity                 int64
weather_main            object
weather_description     object
clouds_all               int64
wind_speed             float64
wind_deg                 int64
gust                   float64
visibility               int64
pop                    float64
sys_pod                 object
dt_txt                  object
city_name               object
city_lat               float64
city_lon               float64
city_country            object
city_timezone           object
city_sunrise            object
city_sunset             object
dtype: object

#### Data Ingestion Pipeline

In [148]:
# Function to fetch and ingest data - compiling everything above into a function
def fetch_and_ingest_data2():
    lat2= 43.6532
    lon2 = -79.3832
    api_key2 = "583b9776c32850d28350562414730cb1"
    api_url2 = 'https://pro.openweathermap.org/data/2.5/forecast/hourly?lat={}&lon={}&appid={}'.format(lat2, lon2, api_key2)
    data2 = requests.get(api_url2)
    data2 = data2.json()
   
    # Flatten the nested data
    flat_data_list = []
    
    for item in data2['list']:
        flat_data = {
        'dt': item['dt'],
        'temp': item['main']['temp'],
        'feels_like': item['main']['feels_like'],
        'temp_min': item['main']['temp_min'],
        'temp_max': item['main']['temp_max'],
        'pressure': item['main']['pressure'],
        'sea_level': item['main'].get('sea_level'),  # Check if key exists
        'grnd_level': item['main'].get('grnd_level'),  # Check if key exists
        'temp_kf': item['main'].get('temp_kf'),  # Check if key exists
        'humidity': item['main']['humidity'],
        'weather_id': item['weather'][0]['id'],
        'weather_main': item['weather'][0]['main'],
        'weather_description': item['weather'][0]['description'],
        'weather_icon': item['weather'][0]['icon'],
        'clouds_all': item['clouds']['all'],
        'wind_speed': item['wind']['speed'],
        'wind_deg': item['wind']['deg'],
        'gust': item['wind']['gust'],
        'visibility': item['visibility'],
        'pop': item['pop'],
        'sys_pod': item['sys']['pod'],
        'dt_txt': item['dt_txt'],
        'city_id': data2['city']['id'],
        'city_name': data2['city']['name'],
        'city_lat': data2['city']['coord']['lat'],
        'city_lon': data2['city']['coord']['lon'],
        'city_country': data2['city']['country'],
        'city_timezone': data2['city']['timezone'],
        'city_sunrise': data2['city']['sunrise'],
        'city_sunset': data2['city']['sunset']
        }
        flat_data_list.append(flat_data)
    
    # Convert the flattened data to a DataFrame
    df2 = pd.DataFrame(flat_data_list)

    # Convert UNIX timestamps to 'hh:mm:ss' format using lambda function
    df2['dt'] = df2['dt'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    df2['city_sunrise'] = df2['city_sunrise'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    df2['city_sunset'] = df2['city_sunset'].apply(lambda unix_time: datetime.utcfromtimestamp(unix_time).strftime('%H:%M:%S'))
    
    # Convert timezone shift to 'hhmm' format using lambda function
    df2['city_timezone'] = df2['city_timezone'].apply(lambda shift: ('-' if shift < 0 else '') + '{:02d}{:02d}'.format(abs(shift) // 3600, (abs(shift) % 3600) // 60))

    #Dropping unneceassry columns and intenal parameters
    df2.drop(['temp_kf','weather_id','weather_icon','city_id'], axis=1, inplace=True)

    #Pushing dataframe to Database Schema
    df2.to_sql(
    name      = 'Toronto_weather_forecasts',
    con       = my_db_engine,
    schema    = 'Uploads',
    if_exists = 'replace',
    index     = False,
    method    = 'multi',
    dtype     = {
        'dt'                  : sa.types.VARCHAR(30),
        'temp'                : sa.types.FLOAT,
        'feels_like'          : sa.types.FLOAT,
        'temp_min'            : sa.types.FLOAT,
        'temp_max'            : sa.types.FLOAT,
        'pressure'            : sa.types.INTEGER,
        'sea_leve'            : sa.types.INTEGER,
        'grnd_level'          : sa.types.INTEGER,
        'humidity'            : sa.types.INTEGER,
        'weather_main'        : sa.types.VARCHAR(30),
        'weather_description' : sa.types.VARCHAR(30),
        'clouds_all'          : sa.types.INTEGER,
        'wind_speed'          : sa.types.FLOAT,
        'wind_deg'            : sa.types.INTEGER,
        'gust'                : sa.types.FLOAT,
        'visibility'          : sa.types.INTEGER,
        'pop'                 : sa.types.FLOAT,
        'sys_pod'             : sa.types.VARCHAR(30),
        'dt_txt'              : sa.types.VARCHAR(30),
        'city_name'           : sa.types.VARCHAR(30),
        'city_lon'            : sa.types.FLOAT,
        'city_lat'            : sa.types.FLOAT, 
        'city_country'        : sa.types.VARCHAR(30),
        'city_timezone'       : sa.types.VARCHAR(30),
        'city_sunrise'        : sa.types.VARCHAR(30),
        'city_sunset'         : sa.types.VARCHAR(30)        
        }
    )

# Immediately run the data fetching and ingestion process
print("Running data fetching and ingestion process immediately...")
fetch_and_ingest_data2()
print("Data fetching and ingestion process has been executed.")

Running data fetching and ingestion process immediately...
Data fetching and ingestion process has been executed.


In [149]:
# Schedule the data fetching and ingestion process to run daily from the next day onwards
# P.S. - This is meant to be executed in the background. 
def schedule_daily_task2(hour, minute):
    while True:
        current_time = time.localtime()
        if current_time.tm_hour >= hour and current_time.tm_min >= minute:
            break
        time.sleep(60)  # Sleep for a minute

    while True:
        fetch_and_ingest_data2()
        print("Data fetching and ingestion process has been scheduled and executed.")
        time.sleep(24 * 60 * 60)  # Sleep for a day

# Schedule the data fetching and ingestion process to run every day at 12:00 PM
schedule_daily_task2(hour=12, minute=0)

Data fetching and ingestion process has been scheduled and executed.
